In [1]:
# model_checkpoint = "bert-base-chinese"
# model_checkpoint = "schen/longformer-chinese-base-4096"
model_checkpoint = "hfl/chinese-xlnet-base"
batch_size = 2 # 每一批次的数量
num_labels = 2 # 多少分类，这里是二分类问题，积极和消极
output_dir = "/home/chenli/pre_model/roberta" # 模型保存路径
learning_rate = 1e-5 # 学习率
# weight_decay=0.01 # 学习率衰减，设置0.01即可。如果weight_decay设置太小，几乎就不起作用了。
num_train_epochs = 10 # 训练轮次，差不多设置为5左右。轮数不要设置太大。轮数设置的太大，Loss是下降了，但是微调的时候效果不是很好，有可能训练过头了

In [2]:
from datasets import load_dataset
from datasets import load_from_disk
# 加载一个评估标准，默认的评估标准
from datasets import load_metric

In [3]:
# 加载数据集
train_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/train_dataset.csv',split='train')
valid_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/valid_dataset.csv',split='train')
test_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/test_dataset.csv',split='train')

Using custom data configuration default-be87a0d28564f2ae
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-be87a0d28564f2ae/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-e6970b6e6097a9c9
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-e6970b6e6097a9c9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-8cb27c720d5c6049
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-8cb27c720d5c6049/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [4]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 33858
})

In [5]:
valid_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4232
})

In [6]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4233
})

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=True)

In [9]:
# 分词
def preprocess_function(data):
    return tokenizer(data['text'],padding='max_length', max_length=100,truncation=True)

In [10]:
train_encoded_dataset = train_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
val_encoded_dataset = valid_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
test_encoded_dataset = test_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])

Loading cached processed dataset at /home/chenli/.cache/huggingface/datasets/csv/default-be87a0d28564f2ae/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-83047abfc340c2a8.arrow


  0%|          | 0/5 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/chenli/.cache/huggingface/datasets/csv/default-8cb27c720d5c6049/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-8630492cb893a15a.arrow


In [11]:
train_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 33858
})

In [12]:
val_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4232
})

In [13]:
test_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4233
})

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [15]:
metric_name = "accuracy"

args = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",# no step epoch
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = 1,
    num_train_epochs = num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # _n_gpu=2
)

In [16]:
import numpy as np
def compute_metrics(eval_preds):
    metric = load_metric('glue','mrpc')
    logits,labels = eval_preds # 预测值和真实值
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_encoded_dataset,
    eval_dataset=val_encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### 20230522

In [18]:
trainer.train()

***** Running training *****
  Num examples = 33858
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84650
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.144400,0.185793,0.965501,0.976722
2,0.122200,0.151080,0.970699,0.980141
3,0.095900,0.163415,0.970463,0.979997
4,0.060300,0.145804,0.969991,0.979838
5,0.038000,0.200502,0.965974,0.977063


***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-8465
Configuration saved in /home/chenli/pre_model/roberta/checkpoint-8465/config.json
Model weights saved in /home/chenli/pre_model/roberta/checkpoint-8465/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/roberta/checkpoint-8465/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/roberta/checkpoint-8465/special_tokens_map.json
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-16930
Configuration saved in /home/chenli/pre_m

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 542427136 vs 542427088

### 20230523

In [18]:
trainer.train()

***** Running training *****
  Num examples = 33858
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84650
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.146000,0.186537,0.964319,0.976051
2,0.110800,0.148461,0.968336,0.978560
3,0.087400,0.180977,0.968336,0.978628
4,0.064900,0.160424,0.970463,0.980080
5,0.042600,0.194908,0.967155,0.977877
6,0.021800,0.302397,0.967155,0.977856
7,0.013300,0.306212,0.969045,0.979124
8,0.008100,0.335201,0.967155,0.977926
9,0.015200,0.310153,0.968573,0.978825
9,0.015200,0.312419,0.968344,0.979289


***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-8465
Configuration saved in /home/chenli/pre_model/roberta/checkpoint-8465/config.json
Model weights saved in /home/chenli/pre_model/roberta/checkpoint-8465/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/roberta/checkpoint-8465/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/roberta/checkpoint-8465/special_tokens_map.json
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Using the latest cached version of the module from /home/chenli/.cache/huggingface/modules/datasets_modules/metrics/

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 542427520 vs 542427472

In [ ]:
model_dir = "/home/chenli/pre_model/roberta/checkpoint-33860"
from transformers import AutoModelForSequenceClassification
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
# 加载原始模型
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels)
model.to(device)
# 启动模型
model.eval()

评估测试

In [19]:
trainer.evaluate(eval_dataset=test_encoded_dataset)

***** Running Evaluation *****
  Num examples = 4233
  Batch size = 2
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Using the latest cached version of the module from /home/chenli/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Nov 11 21:04:53 2022) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


{'eval_loss': 0.31241855025291443,
 'eval_accuracy': 0.9683439640916608,
 'eval_f1': 0.9792890262751159}